# Initial Setup

In [ ]:
!pip install "tf-models-official"

In [17]:
import os
import io
import pprint
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from six import BytesIO
from IPython import display
from urllib.request import urlopen

In [18]:
import orbit
import tensorflow_models as tfm

from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

pp = pprint.PrettyPrinter(indent=4) # Set Pretty Print Indentation
print(tf.__version__) # Check the version of tensorflow used

%matplotlib inline

2.16.1


# LabelMaps

## Creacion de Labelmaps

In [ ]:
!python ./csv_to_labelmap.py

# TFRecords

## Train Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/train/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/train" train.record

## Test Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/test/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/test" test.record

## Valid Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/valid/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/valid" valid.record

# MLModel

## Libraries

In [19]:
import wget
import tarfile
import os
import re

## Enviroment Settings

In [20]:
#Can be configured
DATASET_DIRECTORY = "/tf/dataset/workspaceCards/"
OBJECT_DETECTION_DIRECTORY = "/tf/tensorflow/models/research/object_detection/"

MODEL_NAME = "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8"
MODEL_LINK = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz"
MODEL_CONFIG = "ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.config"
MODELS_FOLDER = OBJECT_DETECTION_DIRECTORY+"configs/tf2/"
MODEL_FILEPATH = os.path.join(MODELS_FOLDER, MODEL_CONFIG)

MODEL_TRAIN = DATASET_DIRECTORY + "trainModel/"
MODEL_EXPORT = DATASET_DIRECTORY + "exportModel/"

In [21]:
FINE_CHECKPOINT = DATASET_DIRECTORY + "models/" + MODEL_NAME + "/checkpoint/cpkt-0"
TRAIN_TFRECORD = DATASET_DIRECTORY + "train.record"
TEST_TFRECORD = DATASET_DIRECTORY + "test.record"
VALID_TFRECORD = DATASET_DIRECTORY + "valid.record"
LABEL_MAP = DATASET_DIRECTORY + "label_map.pbtxt"

In [22]:
BATCH_SIZE = 4
NUM_STEPS = 10000
NUM_CLASSES = 52
NUM_EVAL_STEPS = 50

In [23]:
exp_config = exp_factory.get_exp_config('retinanet_resnetfpn_coco')

HEIGHT, WIDTH = 640, 640
IMG_SIZE = [HEIGHT, WIDTH, 3]

# Backbone config.
exp_config.task.freeze_backbone = False
exp_config.task.annotation_file = ''

# Model config.
exp_config.task.model.input_size = IMG_SIZE
exp_config.task.model.num_classes = NUM_CLASSES
exp_config.task.model.detection_generator.tflite_post_processing.max_classes_per_detection = exp_config.task.model.num_classes

# Training data config.
exp_config.task.train_data.input_path = TRAIN_TFRECORD
exp_config.task.train_data.dtype = 'float32'
exp_config.task.train_data.global_batch_size = BATCH_SIZE
exp_config.task.train_data.parser.aug_scale_max = 1.0
exp_config.task.train_data.parser.aug_scale_min = 1.0

# Validation data config.
exp_config.task.validation_data.input_path = VALID_TFRECORD
exp_config.task.validation_data.dtype = 'float32'
exp_config.task.validation_data.global_batch_size = BATCH_SIZE


In [24]:
logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]

if 'GPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'GPU'
elif 'TPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'TPU'
else:
  print('Running on CPU is slow, so only train for a few steps.')
  device = 'CPU'


train_steps = 1000
exp_config.trainer.steps_per_loop = 100 # steps_per_loop = num_of_training_examples // train_batch_size

exp_config.trainer.summary_interval = 100
exp_config.trainer.checkpoint_interval = 100
exp_config.trainer.validation_interval = 100
exp_config.trainer.validation_steps =  100 # validation_steps = num_of_validation_examples // eval_batch_size
exp_config.trainer.train_steps = train_steps
exp_config.trainer.optimizer_config.warmup.linear.warmup_steps = 100
exp_config.trainer.optimizer_config.learning_rate.type = 'cosine'
exp_config.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
exp_config.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.1
exp_config.trainer.optimizer_config.warmup.linear.warmup_learning_rate = 0.05


Running on CPU is slow, so only train for a few steps.


In [25]:
if exp_config.runtime.mixed_precision_dtype == tf.float16:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

if 'GPU' in ''.join(logical_device_names):
  distribution_strategy = tf.distribute.MirroredStrategy()
elif 'TPU' in ''.join(logical_device_names):
  tf.tpu.experimental.initialize_tpu_system()
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='/device:TPU_SYSTEM:0')
  distribution_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  print('Warning: this will be really slow.')
  distribution_strategy = tf.distribute.OneDeviceStrategy(logical_device_names[0])

print('Done')

Done


In [26]:
with distribution_strategy.scope():
  task = tfm.core.task_factory.get_task(exp_config.task, logging_dir=MODEL_TRAIN)


In [27]:
for images, labels in task.build_inputs(exp_config.task.train_data).take(1):
  print()
  print(f'images.shape: {str(images.shape):16}  images.dtype: {images.dtype!r}')
  print(f'labels.keys: {labels.keys()}')


images.shape: (4, 640, 640, 3)  images.dtype: tf.float32
labels.keys: dict_keys(['cls_targets', 'box_targets', 'anchor_boxes', 'cls_weights', 'box_weights', 'image_info'])


2024-05-12 00:10:36.312959: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [28]:
buffer_size = 20
num_of_examples = 3

raw_records = tf.data.TFRecordDataset(
    exp_config.task.train_data.input_path).shuffle(
        buffer_size=buffer_size).take(num_of_examples)

In [ ]:
model, eval_logs = tfm.core.train_lib.run_experiment(
    distribution_strategy=distribution_strategy,
    task=task,
    mode='train_and_eval',
    params=exp_config,
    model_dir=MODEL_TRAIN,
    run_post_eval=True)

restoring or initializing model...
restored model from /tf/dataset/workspaceCards/trainModel/ckpt-300.
restored from checkpoint: /tf/dataset/workspaceCards/trainModel/ckpt-300
train | step:    300 | training until step 400...


2024-05-12 00:10:48.024764: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [ ]:
export_saved_model_lib.export_inference_graph(
    input_type='image_tensor',
    batch_size=1,
    input_image_size=[HEIGHT, WIDTH],
    params=exp_config,
    checkpoint_path=tf.train.latest_checkpoint(MODEL_TRAIN),
    export_dir=MODEL_EXPORT)